### 마이데이터를 활용하여 특정 고객 추출 후 기프티쇼 API 활용하여 기프티콘 발송

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
import pymysql
import paramiko
from sshtunnel import SSHTunnelForwarder

#from datetime import datetime, date, timedelta

pkeyfilepath = '/Users/apple/Desktop/ssh_key.pem' ######################### ssh_key.pem 파일 경로 넣으시면 됩니다.
mypkey = paramiko.RSAKey.from_private_key_file(pkeyfilepath)
sql_hostname = 'db-8j8d2-fkr.cdb.fin-ntruss.com'
sql_username = 'wkdtjdgus2142'            ######################### sql_username 입력하시면 됩니다.
sql_password = '?!0hLtR$_i8onochu=aC'            ######################### sql_password 입력하시면 됩니다.
sql_main_database = 'betterday_db'
sql_port = 3306
ssh_host = '10.0.1.14'
ssh_user = 'wkdtjdgus2142'                ######################### ssh_user 입력하시면 됩니다.
ssh_port = 22

# soslSuregi97o#rop0n0

In [3]:
with SSHTunnelForwarder(
            (ssh_host, ssh_port),
            ssh_username=ssh_user,
            ssh_pkey=mypkey,
            remote_bind_address=(sql_hostname, sql_port)) as tunnel:
            con = pymysql.connect(host='127.0.0.1', user=sql_username,
                                  passwd=sql_password, db=sql_main_database,
                                  port=tunnel.local_bind_port)

            #BCID = user_id
            sql = f"""
                SELECT cust_id
                FROM betterday_admin.bo_cust_evnts
                where evnt_id ='2209002' and is_cmplt = 'Y' and is_trm_agr = 'Y'
                and date_format(convert_tz( created_at,'+00:00','+09:00'),'%Y%m%d') < '20221101' ;
            """
            temp = pd.read_sql(sql, con)
temp

,cust_id
0,7706
1,7716
2,7710
3,7722
4,7728
...,...
372,8856
373,8859
374,8860
375,8877


In [4]:
import requests
import json
import ssl

url = 'https://app.betterday.co.kr/api/v1/admin/login'
headers = {"Content-Type": "application/json; charset=UTF-8" }
datas = {
      "id": "admin",
      "password": "finset@01"
}
request_login = requests.post(url,headers=headers,verify=False,data=json.dumps(datas))
token = json.loads(request_login.content)["token"]["access_token"]

In [5]:
array_test = temp['cust_id'].to_list()

In [ ]:
#array_test=[268,10]
url = 'https://app.betterday.co.kr/api/v1/admin/users'
headers = {
    'Content-Type': 'application/json',
    'Authorization' : 'Bearer '+ token
}
datas = {
    "users": array_test
}
request_real = requests.post(url,headers=headers,verify=False,data=json.dumps(datas))
a = json.loads(request_real.content)
df = pd.DataFrame(a)
df.to_csv('event_list_1102.csv')
df

In [9]:
set(temp['cust_id'].to_list()) - set(df['ID'].to_list())

{'7706', '7716', '7770', '8613', '8887'}

In [ ]:
df = pd.read_csv('event_list_1102.csv', index_col=0, dtype=str)
df

In [11]:
len(df['MOBILE_NO'].to_list())

362

In [19]:
now = datetime.now()
now_time = str(now.hour)+str(now.minute)
now_day = str(now.year)+str(now.month)+str(now.day)


api_code = '0204'   # API코드
custom_auth_code = 'REALe175d367815b4d6a8bd160191e8c45f6' # API 인증 키 
custom_auth_token = 'h0kjZn5SyztxvrbGflhmXg==' # API 인증 토큰
dev_yn = 'N' # 테스트여부 설정 값 - yes or no # 운영(N)으로만 설정해서 테스트 해야 합니다.
goods_code = 'G00000760670' # 상품코드
# mms_msg = 'test' # MMS 메시지
# mms_title = 'title_te' # MMS제목 - mms_title 은 10자가 넘으면 안됩니다.
mms_msg = '베러 웰컴 이벤트 당첨을 진심으로 축하드립니다. 베러가 제공하는 금융건강진단과 성향별 투자 포트폴리오 추천으로 더 나은 내일을 만나세요!' # MMS 메시지
mms_title = '베러 자산연결 이벤트 경품' # MMS제목 - mms_title 은 10자가 넘으면 안됩니다.
callbank_no = '15662231' # '-'를 제외한 발신번호 
#phone_no = ['01031399129','01077742140'] # '-'를 제외한 수신번호
phone_no = df['MOBILE_NO'].to_list() # '-'를 제외한 수신번호

In [20]:
user_id = 'admin@finset.io' # 회원 ID - 고객의 unique id인지, 계정 id인지 
gubun = 'N' # MMS발송 구분자 - Y: 핀번호수신 - N: MMS - I: 바코드이미지수신
# 옵션 파라미터
order_no = '' # 주문번호
rev_info_yn = '' # 예약발송여부 (Y:예약, N:실시간)
rev_info_date = '' # 예약일자 (yyyyMMdd)
rev_info_time = '' # 예약시간 (HHmm)
template_id = '' # 카드 아이디
banner_id = '202006010058067' # 배너 아이디

URL = 'https://bizapi.giftishow.com/bizApi/send'

In [24]:
len(phone_no)

363

In [7]:
tr_id_lst = []
for i in range(len(phone_no)):
  tr_id = 'service_' + now_day + '_' + now_time + '_' + str(i)

  input_data = {
    "api_code": f'{api_code}',
    "custom_auth_code": f'{custom_auth_code}',
    "custom_auth_token": f'{custom_auth_token}',
    "dev_yn": f'{dev_yn}',
    "goods_code": f'{goods_code}',
    "mms_msg": f'{mms_msg}',
    "mms_title": f'{mms_title}',
    "callback_no": f'{callbank_no}',
    "phone_no": phone_no[i],
    "tr_id": f'{tr_id}',
    "user_id": f'{user_id}',
    "gubun": f'{gubun}'
  }
  response = requests.post(URL, data=input_data)
  print('발송할 고객 전화번호 - ', phone_no)
  print('')
  print("response: ", response)
  print("response.text: ", response.text)
  tr_id_lst.append(tr_id)

print(tr_id_lst)

발송할 고객 전화번호 - []


In [27]:
len(tr_id_lst)

363

In [28]:
df['tr_id'] = tr_id_lst[:-1]
df.to_csv('event_trid_1102.csv')

In [35]:
tr_id_lst[-1]

'service_2022112_183362'